This notebook trains and tests an SCA agent on the Laser Hockey Enviroment

In [6]:
# Imports
import numpy as np
import laserhockey.hockey_env as h_env
from importlib import reload
from SAC import Agent
import matplotlib.pyplot as plt
# from gym import wrappers

In [7]:
# Global variables
train_games = 500
test_games = 100
load_checkpoint = False

In [8]:
# Initialization 
np.set_printoptions(suppress=True)
reload(h_env)

# get enviroment
env = h_env.HockeyEnv(mode=h_env.HockeyEnv.TRAIN_DEFENSE)

#init agent
agent = Agent(input_dims=env.observation_space.shape, 
              env=env,
              n_actions=int(env.action_space.shape[0]/2))

# and oponent
opponent = h_env.BasicOpponent()


/Users/emilbreustedt/anaconda3/envs/RL/lib/python3.8/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment Hockey-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/emilbreustedt/anaconda3/envs/RL/lib/python3.8/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment Hockey-One-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [9]:
# Plotting function
def plot_learning_curve(x, scores, figure_file):
    running_avg = np.zeros(len(scores))
    for i in range(len(running_avg)):
        running_avg[i] = np.mean(scores[max(0, i-100):(i+1)])
    plt.plot(x, running_avg)
    plt.title('Running average of previous 100 scores')
    plt.savefig(figure_file)

In [10]:
# Training of agent

# uncomment this line and do a mkdir tmp && mkdir video if you want to
# record video of the agent playing the game.
#env = wrappers.Monitor(env, 'tmp/video', video_callable=lambda episode_id: True, force=True)

best_score = env.reward_range[0]
score_history = []

if not load_checkpoint:
    for i in range(train_games):
        
        # set to normal mode after 50 games
        if train_games == 100: 
            env = h_env.HockeyEnv(mode=h_env.HockeyEnv.NORMAL)

        observation, info = env.reset()
        obs_opponent = env.obs_agent_two()
        
        done = False
        score = 0
        
        while not done:
            action1 = agent.act(observation)
            action2 = opponent.act(obs_opponent)
            
            observation_, reward, done, _, info = env.step(np.hstack([action1,action2]))
            reward = reward
            score += reward
            agent.remember(observation, action1, reward, observation_, done)
            if train_games > 10:
                agent.learn()
                
            obs_opponent = env.obs_agent_two()
            observation = observation_
            
        score_history.append(score)
        avg_score = np.mean(score_history[-100:])

        if avg_score > best_score and train_games > 10:
            best_score = avg_score
            if not load_checkpoint:
                agent.save_models()

        print('episode ', i, 'score %.1f' % score, 'avg_score %.1f' % avg_score)

    x = [i+1 for i in range(train_games)]
    plot_learning_curve(x, score_history, 'SCA.png')
'''
else: 
    agent.load_models()
    player1 = h_env.HumanOpponent(env=env, player=1)
    player2 = agent
    env.render()   
    obs, info = env.reset()
    obs_agent2 = env.obs_agent_two()
    for _ in range(5000):
        env.render()
        a1 = player1.act(obs) 
        a2 = player2.act(obs_agent2)
        obs, r, d, _, info = env.step(np.hstack([a1,a2]))    
        obs_agent2 = env.obs_agent_two()
        if d: break
        '''


.... saving models ....
episode  0 score -6.0 avg_score -6.0
episode  1 score -16.3 avg_score -11.1
episode  2 score -12.4 avg_score -11.6
episode  3 score -13.0 avg_score -11.9
episode  4 score -13.4 avg_score -12.2
episode  5 score -18.9 avg_score -13.3
episode  6 score -7.9 avg_score -12.5
episode  7 score -8.3 avg_score -12.0
episode  8 score 3.4 avg_score -10.3
episode  9 score -13.1 avg_score -10.6
episode  10 score -19.2 avg_score -11.4
episode  11 score -7.8 avg_score -11.1
episode  12 score -12.9 avg_score -11.2
episode  13 score -17.1 avg_score -11.6
episode  14 score -16.5 avg_score -11.9
episode  15 score 8.4 avg_score -10.7
episode  16 score -16.5 avg_score -11.0
episode  17 score -12.2 avg_score -11.1
episode  18 score -19.0 avg_score -11.5
episode  19 score 2.6 avg_score -10.8
episode  20 score -12.5 avg_score -10.9
episode  21 score -12.4 avg_score -11.0
episode  22 score -16.2 avg_score -11.2
episode  23 score -12.9 avg_score -11.3
episode  24 score 8.1 avg_score -10.5

In [ ]:
# Testing

wins = 0
agent.load_models()
player1 = agent
player2 = h_env.BasicOpponent()

for i in range(test_games):
    observation, info = env.reset()
    obs_opponent = env.obs_agent_two()
    done = False
    while not done:
        action1 = agent.act(observation)
        action2 = opponent.act(obs_opponent)

        observation, reward, done, _, info = env.step(np.hstack([action1,action2]))

        obs_opponent = env.obs_agent_two()
    if info['winner']==1:
            wins+=1
        
wins

.... loading models ....


35